In [1]:
import glob
import pandas as pd
import numpy as np

import os
import os.path as osp
import re

In [2]:
attack = "fgsm"

In [3]:
base_path = osp.join(os.getcwd(),"results")
attack_folder =  f"attack_{attack}"
seeds = [str(i*111) for i in range(1,6)]
results_folder = os.path.join(base_path, attack_folder)

In [4]:
algorithms = ["draft","noise","fgsm","pgd","aae","crownibp"]
exclude_datasets = ["Dialysis","divorce","Pbc3","vlbw"]

In [31]:
# CI , IBS , NegLL
metric = "NegLL"

ascending = False if metric=="CI" else True

In [32]:
def get_metric_excel_paths(base_path,metric):
    # Use glob to match all CI.xlsx files in seed_{number} folders
    pattern = os.path.join(base_path, "results_*", "*", "seed_*", f"{metric}.xlsx")
    metric_excel_paths = glob.glob(pattern)
    return metric_excel_paths

def read_metric_data(file_path):
    # Read the CI.xlsx file and extract epsilon and CI values
    df = pd.read_excel(file_path)
    if df.shape[1] != 2:
        raise ValueError(f"Unexpected format in {file_path}. Expected exactly two columns.")

    metric_name = df.columns[-1]
    df.columns = ['epsilon', metric_name]
    return df

def extract_metadata_from_path(path):
    # Extract dataset, algorithm, and attack method from the path
    parts = path.split(os.sep)
    attack_method = parts[-5].replace("attack_", "")
    algorithm = parts[-4].replace("results_", "")
    dataset = parts[-3]
    seed = int(parts[-2].replace("seed_", ""))
    return dataset, algorithm, attack_method, seed

def create_aggregated_dataframe(base_path,metric="CI"):
    # Get all CI.xlsx file paths
    ci_excel_files = get_metric_excel_paths(base_path,metric)
    
    # Dictionary to store dataframes by (dataset, algorithm) keys
    data_dict = {}
    
    # Process each CI.xlsx file
    for file_path in ci_excel_files:
        dataset, algorithm, attack_method, seed = extract_metadata_from_path(file_path)
        metric_data = read_metric_data(file_path)
        
        # Use (dataset, algorithm) as key
        key = (dataset, algorithm)
        
        # Initialize list for the key if not present
        if key not in data_dict:
            data_dict[key] = []
        
        # Append CI data to the list for that key
        data_dict[key].append(metric_data.set_index('epsilon'))
    
    # Dictionary to store aggregated dataframes
    aggregated_data = {}
    
    # Aggregate by dataset and algorithm
    for (dataset, algorithm), dfs in data_dict.items():
        # Concatenate along the columns to align by epsilon values and compute mean
        concatenated_df = pd.concat(dfs, axis=1)
        aggregated_df = concatenated_df.mean(axis=1).to_frame(name=(dataset, algorithm))
        aggregated_data[(dataset, algorithm)] = aggregated_df
    
    # Combine all aggregated dataframes into a single dataframe with multi-index columns
    final_df = pd.concat(aggregated_data.values(), axis=1)

    # Sort the columns by dataset and then by algorithm for a clean MultiIndex
    final_df = final_df.sort_index(axis=1, level=[0, 1])

    # Create a MultiIndex for the columns with levels: dataset and algorithm
    columns = pd.MultiIndex.from_tuples(final_df.columns, names=['Dataset', 'Algorithm'])
    final_df.columns = columns
    
    return final_df


In [33]:
final_df = create_aggregated_dataframe(results_folder,metric).reindex(columns=algorithms, level=1)

In [34]:
rename_dict = {"draft":"DRAFT","noise":"Noise","fgsm":"FGSM","pgd":"PGD","aae":"AAE-Cox","crownibp":"SAWAR"}
algorithms_renamed = list(rename_dict.values())

In [35]:
final_df = final_df.rename(columns=rename_dict, level=1)

In [36]:
final_df

Dataset          Aids2                                                  \
Algorithm        DRAFT       Noise        FGSM         PGD     AAE-Cox   
epsilon                                                                  
1.00       1041.316479  994.145178  615.722742  619.562268  699.656409   
0.90        994.570898  952.794373  609.780359  612.554907  683.244702   
0.80        947.949841  911.021265  603.509253  605.398645  666.991077   
0.70        901.362170  868.779041  597.041956  598.112280  650.830554   
0.60        854.274292  825.928711  590.507056  590.727454  634.803430   
0.50        806.053662  782.160962  583.971484  583.246448  618.887512   
0.40        756.371924  736.623682  577.289148  575.807385  602.928369   
0.30        704.336230  689.147437  570.364832  568.148779  586.868103   
0.20        648.526733  638.556873  562.695288  560.401282  570.696143   
0.10        592.224792  588.153577  553.739355  552.090771  554.589197   
0.05        565.864661  564.149036  548.150745  547.335107  546.651233   
0.00        540.764026  540.864612  541.003662  541.464062  538.844177   

Dataset                 Framingham                                         \
Algorithm       SAWAR        DRAFT        Noise         FGSM          PGD   
epsilon                                                                     
1.00       584.175708  4234.772510  3999.005713  2109.742969  1990.391357   
0.90       580.093958  4104.064404  3892.288818  2068.731812  1953.087378   
0.80       575.987622  3910.868701  3732.006445  2026.288452  1914.731714   
0.70       571.895618  3649.710400  3511.218848  1981.518433  1874.566187   
0.60       567.717957  3330.803711  3232.064990  1933.403662  1831.644507   
0.50       563.439087  2972.653174  2908.741504  1880.334570  1785.603076   
0.40       559.087952  2605.136475  2570.079395  1819.531543  1735.968335   
0.30       554.536121  2255.784424  2241.872656  1749.927661  1681.532373   
0.20       549.860388  1945.872119  1943.160742  1670.373535  1621.411426   
0.10       545.125171  1684.943579  1687.331689  1580.971436  1555.081250   
0.05       542.740698  1574.553296  1577.864868  1532.838477  1519.408423   
0.00       540.356177  1477.818848  1481.085962  1482.796680  1481.939331   

Dataset    ...      stagec                                          zinc  \
Algorithm  ...        FGSM         PGD    AAE-Cox      SAWAR       DRAFT   
epsilon    ...                                                             
1.00       ...  103.975461  100.679590  79.962839  94.608881  501.248212   
0.90       ...  100.569945   97.287720  74.692029  89.758931  494.059882   
0.80       ...   96.250572   93.122528  69.877760  84.137804  476.701514   
0.70       ...   91.052477   88.186751  65.485036  77.959090  444.681995   
0.60       ...   85.195869   82.694754  61.459522  71.681260  392.882507   
0.50       ...   78.514198   76.513837  57.714616  65.736958  319.375012   
0.40       ...   71.206261   69.712938  54.162621  60.264929  239.119635   
0.30       ...   63.688446   62.675648  50.737012  54.827353  172.988470   
0.20       ...   56.399445   55.780368  47.372803  49.515571  126.794171   
0.10       ...   49.632972   49.310841  44.108498  44.539946   96.178517   
0.05       ...   46.494688   46.295885  42.492518  42.262185   85.241406   
0.00       ...   43.575211   43.487480  40.909048  40.141538   76.398311   

Dataset                                                                
Algorithm       Noise        FGSM         PGD     AAE-Cox       SAWAR  
epsilon                                                                
1.00       554.428412  288.656943  277.215573  165.891089  211.199451  
0.90       533.549915  279.089017  267.418842  156.827734  191.773441  
0.80       499.976080  262.831061  251.445731  147.576274  173.204691  
0.70       451.567212  239.040289  228.780673  137.687956  156.566211  
0.60       387.368842  209.806512  201.699210  127.338406  141.628006  
0.50       310.1

In [37]:
final_df[['zinc']]

Dataset          zinc                                                  \
Algorithm       DRAFT       Noise        FGSM         PGD     AAE-Cox   
epsilon                                                                 
1.00       501.248212  554.428412  288.656943  277.215573  165.891089   
0.90       494.059882  533.549915  279.089017  267.418842  156.827734   
0.80       476.701514  499.976080  262.831061  251.445731  147.576274   
0.70       444.681995  451.567212  239.040289  228.780673  137.687956   
0.60       392.882507  387.368842  209.806512  201.699210  127.338406   
0.50       319.375012  310.105383  179.154996  173.586060  117.238199   
0.40       239.119635  235.407767  151.597446  147.879996  107.906032   
0.30       172.988470  175.393582  128.201283  125.938133   99.418373   
0.20       126.794171  131.841522  108.659569  107.441220   91.632487   
0.10        96.178517  100.522995   92.141443   91.753802   84.396274   
0.05        85.241406   88.511940   84.838455   84.833562   80.966156   
0.00        76.398311   78.628049   78.230508   78.482152   77.644135   

Dataset                
Algorithm       SAWAR  
epsilon                
1.00       211.199451  
0.90       191.773441  
0.80       173.204691  
0.70       156.566211  
0.60       141.628006  
0.50       127.951086  
0.40       115.498515  
0.30       104.200539  
0.20        94.196269  
0.10        85.195714  
0.05        81.064696  
0.00        77.195442

In [38]:
final_df[final_df==''] = np.NaN
final_df = final_df.astype(float)
final_df

Dataset          Aids2                                                  \
Algorithm        DRAFT       Noise        FGSM         PGD     AAE-Cox   
epsilon                                                                  
1.00       1041.316479  994.145178  615.722742  619.562268  699.656409   
0.90        994.570898  952.794373  609.780359  612.554907  683.244702   
0.80        947.949841  911.021265  603.509253  605.398645  666.991077   
0.70        901.362170  868.779041  597.041956  598.112280  650.830554   
0.60        854.274292  825.928711  590.507056  590.727454  634.803430   
0.50        806.053662  782.160962  583.971484  583.246448  618.887512   
0.40        756.371924  736.623682  577.289148  575.807385  602.928369   
0.30        704.336230  689.147437  570.364832  568.148779  586.868103   
0.20        648.526733  638.556873  562.695288  560.401282  570.696143   
0.10        592.224792  588.153577  553.739355  552.090771  554.589197   
0.05        565.864661  564.149036  548.150745  547.335107  546.651233   
0.00        540.764026  540.864612  541.003662  541.464062  538.844177   

Dataset                 Framingham                                         \
Algorithm       SAWAR        DRAFT        Noise         FGSM          PGD   
epsilon                                                                     
1.00       584.175708  4234.772510  3999.005713  2109.742969  1990.391357   
0.90       580.093958  4104.064404  3892.288818  2068.731812  1953.087378   
0.80       575.987622  3910.868701  3732.006445  2026.288452  1914.731714   
0.70       571.895618  3649.710400  3511.218848  1981.518433  1874.566187   
0.60       567.717957  3330.803711  3232.064990  1933.403662  1831.644507   
0.50       563.439087  2972.653174  2908.741504  1880.334570  1785.603076   
0.40       559.087952  2605.136475  2570.079395  1819.531543  1735.968335   
0.30       554.536121  2255.784424  2241.872656  1749.927661  1681.532373   
0.20       549.860388  1945.872119  1943.160742  1670.373535  1621.411426   
0.10       545.125171  1684.943579  1687.331689  1580.971436  1555.081250   
0.05       542.740698  1574.553296  1577.864868  1532.838477  1519.408423   
0.00       540.356177  1477.818848  1481.085962  1482.796680  1481.939331   

Dataset    ...      stagec                                          zinc  \
Algorithm  ...        FGSM         PGD    AAE-Cox      SAWAR       DRAFT   
epsilon    ...                                                             
1.00       ...  103.975461  100.679590  79.962839  94.608881  501.248212   
0.90       ...  100.569945   97.287720  74.692029  89.758931  494.059882   
0.80       ...   96.250572   93.122528  69.877760  84.137804  476.701514   
0.70       ...   91.052477   88.186751  65.485036  77.959090  444.681995   
0.60       ...   85.195869   82.694754  61.459522  71.681260  392.882507   
0.50       ...   78.514198   76.513837  57.714616  65.736958  319.375012   
0.40       ...   71.206261   69.712938  54.162621  60.264929  239.119635   
0.30       ...   63.688446   62.675648  50.737012  54.827353  172.988470   
0.20       ...   56.399445   55.780368  47.372803  49.515571  126.794171   
0.10       ...   49.632972   49.310841  44.108498  44.539946   96.178517   
0.05       ...   46.494688   46.295885  42.492518  42.262185   85.241406   
0.00       ...   43.575211   43.487480  40.909048  40.141538   76.398311   

Dataset                                                                
Algorithm       Noise        FGSM         PGD     AAE-Cox       SAWAR  
epsilon                                                                
1.00       554.428412  288.656943  277.215573  165.891089  211.199451  
0.90       533.549915  279.089017  267.418842  156.827734  191.773441  
0.80       499.976080  262.831061  251.445731  147.576274  173.204691  
0.70       451.567212  239.040289  228.780673  137.687956  156.566211  
0.60       387.368842  209.806512  201.699210  127.338406  141.628006  
0.50       310.1

In [39]:
final_df.applymap(np.isnan).sum().sum()

C:\Users\lpott\AppData\Local\Temp\ipykernel_440\2774236446.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  final_df.applymap(np.isnan).sum().sum()


0

In [40]:
best_per_dataset_rank = final_df.groupby(level=0,axis=1).rank(axis=1,na_option='bottom',method="average",ascending=ascending).reindex(columns=algorithms_renamed, level=1)
best_per_dataset_rank

C:\Users\lpott\AppData\Local\Temp\ipykernel_440\3915878950.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  best_per_dataset_rank = final_df.groupby(level=0,axis=1).rank(axis=1,na_option='bottom',method="average",ascending=ascending).reindex(columns=algorithms_renamed, level=1)
C:\Users\lpott\AppData\Local\Temp\ipykernel_440\3915878950.py:1: FutureWarning: DataFrameGroupBy.rank with axis=1 is deprecated and will be removed in a future version. Operate on the un-grouped DataFrame instead
  best_per_dataset_rank = final_df.groupby(level=0,axis=1).rank(axis=1,na_option='bottom',method="average",ascending=ascending).reindex(columns=algorithms_renamed, level=1)


Dataset   Aids2                               Framingham                  ...  \
Algorithm DRAFT Noise FGSM  PGD AAE-Cox SAWAR      DRAFT Noise FGSM  PGD  ...   
epsilon                                                                   ...   
1.00        6.0   5.0  2.0  3.0     4.0   1.0        6.0   5.0  3.0  2.0  ...   
0.90        6.0   5.0  2.0  3.0     4.0   1.0        6.0   5.0  3.0  2.0  ...   
0.80        6.0   5.0  2.0  3.0     4.0   1.0        6.0   5.0  3.0  2.0  ...   
0.70        6.0   5.0  2.0  3.0     4.0   1.0        6.0   5.0  3.0  2.0  ...   
0.60        6.0   5.0  2.0  3.0     4.0   1.0        6.0   5.0  3.0  2.0  ...   
0.50        6.0   5.0  3.0  2.0     4.0   1.0        6.0   5.0  4.0  2.0  ...   
0.40        6.0   5.0  3.0  2.0     4.0   1.0        6.0   5.0  4.0  2.0  ...   
0.30        6.0   5.0  3.0  2.0     4.0   1.0        6.0   5.0  4.0  2.0  ...   
0.20        6.0   5.0  3.0  2.0     4.0   1.0        6.0   5.0  4.0  3.0  ...   
0.10        6.0   5.0  3.0  2.0     4.0   1.0        5.0   6.0  4.0  3.0  ...   
0.05        6.0   5.0  4.0  3.0     2.0   1.0        5.0   6.0  4.0  3.0  ...   
0.00        3.0   4.0  5.0  6.0     1.0   2.0        3.0   4.0  6.0  5.0  ...   

Dataset   stagec                     zinc                                
Algorithm   FGSM  PGD AAE-Cox SAWAR DRAFT Noise FGSM  PGD AAE-Cox SAWAR  
epsilon                                                                  
1.00         4.0  3.0     1.0   2.0   5.0   6.0  4.0  3.0     1.0   2.0  
0.90         4.0  3.0     1.0   2.0   5.0   6.0  4.0  3.0     1.0   2.0  
0.80         4.0  3.0     1.0   2.0   5.0   6.0  4.0  3.0     1.0   2.0  
0.70         4.0  3.0     1.0   2.0   5.0   6.0  4.0  3.0     1.0   2.0  
0.60         4.0  3.0     1.0   2.0   6.0   5.0  4.0  3.0     1.0   2.0  
0.50         4.0  3.0     1.0   2.0   6.0   5.0  4.0  3.0     1.0   2.0  
0.40         4.0  3.0     1.0   2.0   6.0   5.0  4.0  3.0     1.0   2.0  
0.30         4.0  3.0     1.0   2.0   5.0   6.0  4.0  3.0     1.0   2.0  
0.20         4.0  3.0     1.0   2.0   5.0   6.0  4.0  3.0     1.0   2.0  
0.10         5.0  4.0     1.0   2.0   5.0   6.0  4.0  3.0     1.0   2.0  
0.05         5.0  4.0     2.0   1.0   5.0   6.0  4.0  3.0     1.0   2.0  
0.00         6.0  5.0     2.0   1.0   1.0   6.0  4.0  5.0     3.0   2.0  

[12 rows x 60 columns]

In [41]:
# best_per_dataset_rank.stack(1)

In [42]:
best_per_dataset_avg_rank = best_per_dataset_rank.stack(level=1).mean(1).unstack(1).sort_values(by="epsilon",ascending=False)
best_per_dataset_avg_rank

C:\Users\lpott\AppData\Local\Temp\ipykernel_440\2621807682.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  best_per_dataset_avg_rank = best_per_dataset_rank.stack(level=1).mean(1).unstack(1).sort_values(by="epsilon",ascending=False)


Algorithm,DRAFT,Noise,FGSM,PGD,AAE-Cox,SAWAR
epsilon,,,,,,
1.00,5.1,5.8,3.6,2.6,2.5,1.4
0.90,5.1,5.8,3.6,2.6,2.5,1.4
0.80,5.1,5.8,3.6,2.6,2.5,1.4
0.70,5.1,5.8,3.6,2.7,2.5,1.3
0.60,5.2,5.7,3.6,2.7,2.5,1.3
0.50,5.2,5.7,3.9,2.6,2.3,1.3
0.40,5.3,5.6,3.9,2.6,2.3,1.3
0.30,5.1,5.7,3.9,2.8,2.2,1.3
0.20,4.9,5.8,4.0,2.9,2.1,1.3


In [43]:
if not ascending:
    best_per_dataset = final_df.groupby(level=0,axis=1).idxmax(1)
else:
    best_per_dataset = final_df.groupby(level=0,axis=1).idxmin(1)
best_per_dataset.applymap(lambda x: x[1])

C:\Users\lpott\AppData\Local\Temp\ipykernel_440\1286510363.py:4: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  best_per_dataset = final_df.groupby(level=0,axis=1).idxmin(1)
C:\Users\lpott\AppData\Local\Temp\ipykernel_440\1286510363.py:4: FutureWarning: DataFrameGroupBy.idxmin with axis=1 is deprecated and will be removed in a future version. Operate on the un-grouped DataFrame instead
  best_per_dataset = final_df.groupby(level=0,axis=1).idxmin(1)
C:\Users\lpott\AppData\Local\Temp\ipykernel_440\1286510363.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  best_per_dataset.applymap(lambda x: x[1])


Dataset,Aids2,Framingham,LeukSurv,TRACE,dataDIVAT1,flchain,prostate,retinopathy,stagec,zinc
epsilon,,,,,,,,,,
1.00,SAWAR,SAWAR,SAWAR,SAWAR,SAWAR,PGD,SAWAR,AAE-Cox,AAE-Cox,AAE-Cox
0.90,SAWAR,SAWAR,SAWAR,SAWAR,SAWAR,PGD,SAWAR,AAE-Cox,AAE-Cox,AAE-Cox
0.80,SAWAR,SAWAR,SAWAR,SAWAR,SAWAR,PGD,SAWAR,AAE-Cox,AAE-Cox,AAE-Cox
0.70,SAWAR,SAWAR,SAWAR,SAWAR,SAWAR,SAWAR,SAWAR,AAE-Cox,AAE-Cox,AAE-Cox
0.60,SAWAR,SAWAR,SAWAR,SAWAR,SAWAR,SAWAR,SAWAR,AAE-Cox,AAE-Cox,AAE-Cox
0.50,SAWAR,SAWAR,SAWAR,SAWAR,SAWAR,SAWAR,SAWAR,AAE-Cox,AAE-Cox,AAE-Cox
0.40,SAWAR,SAWAR,SAWAR,SAWAR,SAWAR,SAWAR,SAWAR,AAE-Cox,AAE-Cox,AAE-Cox
0.30,SAWAR,SAWAR,SAWAR,SAWAR,SAWAR,SAWAR,SAWAR,AAE-Cox,AAE-Cox,AAE-Cox
0.20,SAWAR,SAWAR,SAWAR,SAWAR,SAWAR,SAWAR,SAWAR,AAE-Cox,AAE-Cox,AAE-Cox


In [44]:
def calculate_percent_change_from_draft(df):
    """
    Calculate the percent change from the 'draft' method for each dataset and algorithm in the given DataFrame.
    
    Args:
        df (pd.DataFrame): MultiIndex DataFrame where level 0 is 'Dataset' and level 1 is 'Algorithm'.
    
    Returns:
        pd.DataFrame: A DataFrame with percent changes from the 'draft' method for each dataset and algorithm.
    """
    # Create a copy of the DataFrame to store the percent changes
    percent_change_df = df.copy()

    # Loop over each dataset in level 0 of the MultiIndex
    for dataset in df.columns.levels[0]:
        # Get the "draft" column for the current dataset
        draft_column = df[(dataset, "DRAFT")]

        # Calculate percent change for each algorithm relative to "draft"
        for algorithm in df.columns.levels[1]:
            # Skip the "draft" column itself as it is 0% change
            if algorithm == "DRAFT":
                continue

            # Calculate percent change and update in the new DataFrame
            percent_change_df[(dataset, algorithm)] = (
                (df[(dataset, algorithm)] - draft_column) / draft_column
            ) * 100  # Multiply by 100 to convert to percentage

    # The "draft" column itself should be 0% change from itself
    for dataset in df.columns.levels[0]:
        percent_change_df[(dataset, "DRAFT")] = 0

    return percent_change_df

In [45]:
percent_change_df = calculate_percent_change_from_draft(final_df).reindex(columns=algorithms_renamed, level=1)

In [46]:
percentage_change_mean =  percent_change_df.stack(level=1).mean(1).unstack(1).sort_values(by="epsilon",ascending=False)
percentage_change_mean

C:\Users\lpott\AppData\Local\Temp\ipykernel_440\211622617.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  percentage_change_mean =  percent_change_df.stack(level=1).mean(1).unstack(1).sort_values(by="epsilon",ascending=False)


Algorithm,DRAFT,Noise,FGSM,PGD,AAE-Cox,SAWAR
epsilon,,,,,,
1.00,0.0,16838.930433,-39.079401,-45.103170,-54.030520,-56.799328
0.90,0.0,11119.163400,-37.957649,-43.941081,-53.751985,-56.645076
0.80,0.0,6645.462938,-36.519958,-42.391373,-52.852684,-55.936997
0.70,0.0,3981.170595,-34.583347,-40.311426,-51.179409,-54.465158
0.60,0.0,2445.039476,-31.936299,-37.517062,-48.577307,-52.021937
0.50,0.0,1454.957561,-28.386593,-33.764250,-44.745758,-48.286004
0.40,0.0,789.779148,-23.676586,-28.789943,-39.276860,-42.856394
0.30,0.0,399.213111,-17.851521,-22.405547,-31.721546,-35.475113
0.20,0.0,196.060768,-10.546257,-14.265194,-22.163015,-25.492787


In [47]:
percentage_change_mean.index = ["{:.2f}".format(float(x)) for x in np.round(best_per_dataset_avg_rank.index.tolist(),2)]
percentage_change_mean.index.name = r"$\epsilon$"
percentage_change_mean[['SAWAR']].T.round(2)

$\epsilon$,1.00,0.90,0.80,0.70,0.60,0.50,0.40,0.30,0.20,0.10,0.05,0.00
Algorithm,,,,,,,,,,,,
SAWAR,-56.8,-56.65,-55.94,-54.47,-52.02,-48.29,-42.86,-35.48,-25.49,-13.63,-7.46,-1.79


In [48]:
percentage_change_mean

Algorithm,DRAFT,Noise,FGSM,PGD,AAE-Cox,SAWAR
$\epsilon$,,,,,,
1.00,0.0,16838.930433,-39.079401,-45.103170,-54.030520,-56.799328
0.90,0.0,11119.163400,-37.957649,-43.941081,-53.751985,-56.645076
0.80,0.0,6645.462938,-36.519958,-42.391373,-52.852684,-55.936997
0.70,0.0,3981.170595,-34.583347,-40.311426,-51.179409,-54.465158
0.60,0.0,2445.039476,-31.936299,-37.517062,-48.577307,-52.021937
0.50,0.0,1454.957561,-28.386593,-33.764250,-44.745758,-48.286004
0.40,0.0,789.779148,-23.676586,-28.789943,-39.276860,-42.856394
0.30,0.0,399.213111,-17.851521,-22.405547,-31.721546,-35.475113
0.20,0.0,196.060768,-10.546257,-14.265194,-22.163015,-25.492787


In [49]:
excel_name = os.path.join(results_folder,f"{metric}_all.xlsx")
with pd.ExcelWriter(excel_name) as writer:  
    final_df.applymap(lambda x: np.round(x,3)).to_excel(writer,sheet_name=metric)
    best_per_dataset_rank.to_excel(writer,sheet_name="rank")
    best_per_dataset_avg_rank.to_excel(writer,sheet_name="average_rank")
    best_per_dataset.applymap(lambda x: x[1]).to_excel(writer,sheet_name="best")
    percentage_change_mean.to_excel(writer,sheet_name="%")

C:\Users\lpott\AppData\Local\Temp\ipykernel_440\1705722331.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  final_df.applymap(lambda x: np.round(x,3)).to_excel(writer,sheet_name=metric)
C:\Users\lpott\AppData\Local\Temp\ipykernel_440\1705722331.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  best_per_dataset.applymap(lambda x: x[1]).to_excel(writer,sheet_name="best")


In [50]:
best_per_dataset_avg_rank.index = ["{:.2f}".format(float(x)) for x in np.round(best_per_dataset_avg_rank.index.tolist(),2)]
best_per_dataset_avg_rank.index.name = r"$\epsilon$"
print(best_per_dataset_avg_rank.applymap(lambda x: str(np.round(x,2))).to_latex(index=True))

\begin{tabular}{lllllll}
\toprule
Algorithm & DRAFT & Noise & FGSM & PGD & AAE-Cox & SAWAR \\
$\epsilon$ &  &  &  &  &  &  \\
\midrule
1.00 & 5.1 & 5.8 & 3.6 & 2.6 & 2.5 & 1.4 \\
0.90 & 5.1 & 5.8 & 3.6 & 2.6 & 2.5 & 1.4 \\
0.80 & 5.1 & 5.8 & 3.6 & 2.6 & 2.5 & 1.4 \\
0.70 & 5.1 & 5.8 & 3.6 & 2.7 & 2.5 & 1.3 \\
0.60 & 5.2 & 5.7 & 3.6 & 2.7 & 2.5 & 1.3 \\
0.50 & 5.2 & 5.7 & 3.9 & 2.6 & 2.3 & 1.3 \\
0.40 & 5.3 & 5.6 & 3.9 & 2.6 & 2.3 & 1.3 \\
0.30 & 5.1 & 5.7 & 3.9 & 2.8 & 2.2 & 1.3 \\
0.20 & 4.9 & 5.8 & 4.0 & 2.9 & 2.1 & 1.3 \\
0.10 & 4.5 & 5.9 & 4.2 & 3.1 & 1.9 & 1.4 \\
0.05 & 4.5 & 5.9 & 4.3 & 3.3 & 1.7 & 1.3 \\
0.00 & 3.0 & 5.4 & 4.9 & 4.2 & 1.8 & 1.7 \\
\bottomrule
\end{tabular}



C:\Users\lpott\AppData\Local\Temp\ipykernel_440\3581541270.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(best_per_dataset_avg_rank.applymap(lambda x: str(np.round(x,2))).to_latex(index=True))


In [51]:
print(best_per_dataset_avg_rank.applymap(lambda x: str(np.round(x,2))).T.to_latex(index=True))

\begin{tabular}{lllllllllllll}
\toprule
$\epsilon$ & 1.00 & 0.90 & 0.80 & 0.70 & 0.60 & 0.50 & 0.40 & 0.30 & 0.20 & 0.10 & 0.05 & 0.00 \\
Algorithm &  &  &  &  &  &  &  &  &  &  &  &  \\
\midrule
DRAFT & 5.1 & 5.1 & 5.1 & 5.1 & 5.2 & 5.2 & 5.3 & 5.1 & 4.9 & 4.5 & 4.5 & 3.0 \\
Noise & 5.8 & 5.8 & 5.8 & 5.8 & 5.7 & 5.7 & 5.6 & 5.7 & 5.8 & 5.9 & 5.9 & 5.4 \\
FGSM & 3.6 & 3.6 & 3.6 & 3.6 & 3.6 & 3.9 & 3.9 & 3.9 & 4.0 & 4.2 & 4.3 & 4.9 \\
PGD & 2.6 & 2.6 & 2.6 & 2.7 & 2.7 & 2.6 & 2.6 & 2.8 & 2.9 & 3.1 & 3.3 & 4.2 \\
AAE-Cox & 2.5 & 2.5 & 2.5 & 2.5 & 2.5 & 2.3 & 2.3 & 2.2 & 2.1 & 1.9 & 1.7 & 1.8 \\
SAWAR & 1.4 & 1.4 & 1.4 & 1.3 & 1.3 & 1.3 & 1.3 & 1.3 & 1.3 & 1.4 & 1.3 & 1.7 \\
\bottomrule
\end{tabular}



C:\Users\lpott\AppData\Local\Temp\ipykernel_440\2619039113.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(best_per_dataset_avg_rank.applymap(lambda x: str(np.round(x,2))).T.to_latex(index=True))


In [52]:
if metric == "NegLL":
    final_df.index = ["{:.2f}".format(float(x)) for x in np.round(final_df.index.tolist(),2)]
    final_df.index.name = r"$\epsilon$"
    print(final_df.applymap(lambda x: np.round(x,3)).applymap(lambda x: "{:.2e}".format(x)).to_latex(index=True,multicolumn_format="c"))
else:
    final_df.index = ["{:.2f}".format(float(x)) for x in np.round(final_df.index.tolist(),2)]
    final_df.index.name = r"$\epsilon$"
    print(final_df.applymap(lambda x: np.round(x,3)).applymap(str).to_latex(index=True,multicolumn_format="c"))

\begin{tabular}{lllllllllllllllllllllllllllllllllllllllllllllllllllllllllllll}
\toprule
Dataset & \multicolumn{6}{c}{Aids2} & \multicolumn{6}{c}{Framingham} & \multicolumn{6}{c}{LeukSurv} & \multicolumn{6}{c}{TRACE} & \multicolumn{6}{c}{dataDIVAT1} & \multicolumn{6}{c}{flchain} & \multicolumn{6}{c}{prostate} & \multicolumn{6}{c}{retinopathy} & \multicolumn{6}{c}{stagec} & \multicolumn{6}{c}{zinc} \\
Algorithm & DRAFT & Noise & FGSM & PGD & AAE-Cox & SAWAR & DRAFT & Noise & FGSM & PGD & AAE-Cox & SAWAR & DRAFT & Noise & FGSM & PGD & AAE-Cox & SAWAR & DRAFT & Noise & FGSM & PGD & AAE-Cox & SAWAR & DRAFT & Noise & FGSM & PGD & AAE-Cox & SAWAR & DRAFT & Noise & FGSM & PGD & AAE-Cox & SAWAR & DRAFT & Noise & FGSM & PGD & AAE-Cox & SAWAR & DRAFT & Noise & FGSM & PGD & AAE-Cox & SAWAR & DRAFT & Noise & FGSM & PGD & AAE-Cox & SAWAR & DRAFT & Noise & FGSM & PGD & AAE-Cox & SAWAR \\
$\epsilon$ &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &

C:\Users\lpott\AppData\Local\Temp\ipykernel_440\1896362681.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(final_df.applymap(lambda x: np.round(x,3)).applymap(lambda x: "{:.2e}".format(x)).to_latex(index=True,multicolumn_format="c"))


In [53]:
dataset_names = np.array(list(map(np.array,final_df.columns)))[:,0]

In [54]:
dataset_names

array(['Aids2', 'Aids2', 'Aids2', 'Aids2', 'Aids2', 'Aids2', 'Framingham',
       'Framingham', 'Framingham', 'Framingham', 'Framingham',
       'Framingham', 'LeukSurv', 'LeukSurv', 'LeukSurv', 'LeukSurv',
       'LeukSurv', 'LeukSurv', 'TRACE', 'TRACE', 'TRACE', 'TRACE',
       'TRACE', 'TRACE', 'dataDIVAT1', 'dataDIVAT1', 'dataDIVAT1',
       'dataDIVAT1', 'dataDIVAT1', 'dataDIVAT1', 'flchain', 'flchain',
       'flchain', 'flchain', 'flchain', 'flchain', 'prostate', 'prostate',
       'prostate', 'prostate', 'prostate', 'prostate', 'retinopathy',
       'retinopathy', 'retinopathy', 'retinopathy', 'retinopathy',
       'retinopathy', 'stagec', 'stagec', 'stagec', 'stagec', 'stagec',
       'stagec', 'zinc', 'zinc', 'zinc', 'zinc', 'zinc', 'zinc'],
      dtype='<U11')

## LONG TABLE

In [55]:
final_df.T

$\epsilon$                     1.00          0.90           0.80  \
Dataset     Algorithm                                              
Aids2       DRAFT      1.041316e+03  9.945709e+02     947.949841   
            Noise      9.941452e+02  9.527944e+02     911.021265   
            FGSM       6.157227e+02  6.097804e+02     603.509253   
            PGD        6.195623e+02  6.125549e+02     605.398645   
            AAE-Cox    6.996564e+02  6.832447e+02     666.991077   
            SAWAR      5.841757e+02  5.800940e+02     575.987622   
Framingham  DRAFT      4.234773e+03  4.104064e+03    3910.868701   
            Noise      3.999006e+03  3.892289e+03    3732.006445   
            FGSM       2.109743e+03  2.068732e+03    2026.288452   
            PGD        1.990391e+03  1.953087e+03    1914.731714   
            AAE-Cox    2.384715e+03  2.269644e+03    2161.771143   
            SAWAR      1.964035e+03  1.902487e+03    1843.395898   
LeukSurv    DRAFT      2.656395e+03  2.164046e+03    1768.995386   
            Noise      1.131309e+06  5.835500e+05  295792.870312   
            FGSM       2.763602e+03  2.330920e+03    1963.853979   
            PGD        1.786275e+03  1.570748e+03    1380.404077   
            AAE-Cox    1.089047e+03  9.833068e+02     886.659009   
            SAWAR      5.395855e+02  5.123666e+02     484.702002   
TRACE       DRAFT      2.309237e+03  2.088921e+03    1866.206543   
            Noise      4.548498e+03  3.847510e+03    3204.677930   
            FGSM       1.226789e+03  1.149568e+03    1073.685449   
            PGD        1.172118e+03  1.102124e+03    1031.459546   
            AAE-Cox    1.070330e+03  1.002594e+03     938.441785   
            SAWAR      1.009561e+03  9.320017e+02     862.874219   
dataDIVAT1  DRAFT      1.783161e+03  1.718407e+03    1641.219629   
            Noise      2.021193e+03  1.959525e+03    1878.935962   
            FGSM       1.085934e+03  1.066677e+03    1046.717236   
            PGD        1.009774e+03  9.945435e+02     978.170435   
            AAE-Cox    1.249410e+03  1.180588e+03    1117.463330   
            SAWAR      9.241571e+02  9.014418e+02     879.448621   
flchain     DRAFT      2.557133e+04  2.355474e+04   21542.107812   
            Noise      1.623947e+05  1.268116e+05   82430.228516   
            FGSM       2.486919e+03  2.168562e+03    1870.220752   
            PGD        2.072925e+03  1.764602e+03    1530.075757   
            AAE-Cox    2.978594e+03  2.852375e+03    2731.580664   
            SAWAR      2.177992e+03  1.832053e+03    1547.618457   
prostate    DRAFT      1.502100e+03  1.441230e+03    1356.692725   
            Noise      1.882679e+06  1.208815e+06  672291.828809   
            FGSM       8.513030e+02  8.484888e+02     841.000195   
            PGD        7.460880e+02  7.323608e+02     717.061853   
            AAE-Cox    4.643985e+02  4.467300e+02     430.664294   
            SAWAR      4.400648e+02  4.276712e+02     415.461713   
retinopathy DRAFT      5.035384e+02  4.933666e+02     469.651904   
            Noise      5.471295e+02  5.288713e+02     496.444641   
            FGSM       4.162483e+02  4.087380e+02     392.867914   
            PGD        4.109247e+02  4.034651e+02     388.091321   
            AAE-Cox    2.266915e+02  2.205267e+02     214.457455   
            SAWAR      3.272103e+02  3.137552e+02     298.636475   
stagec      DRAFT      1.376341e+02  1.315075e+02     124.126949   
            Noise      1.516192e+02  1.442463e+02     135.271191   
            FGSM       1.039755e+02  1.005699e+02      96.250572   
            PGD        1.006796e+02  9.728772e+01      93.122528   
            AAE-Cox    7.996284e+01  7.469203e+01      69.877760   
            SAWAR      9.460888e+01  8.975893e+01      84.137804   
zinc        DRAFT      5.012482e+02  4.940599e+02     476.701514   
            Noise      5.544284e+02  5.335499e+02     499.976080   
            FGSM       2.886569e+02  2.790890e+02     26

In [56]:
if metric == "NegLL":
    print(final_df.T.applymap(lambda x: str(np.round(x,3))).applymap(lambda x: "{:.2e}".format(float(x))).to_latex(index=True,multicolumn_format="c"))
else:
    print(final_df.T.applymap(lambda x: str(np.round(x,3))).to_latex(index=True,multicolumn_format="c"))

\begin{tabular}{llllllllllllll}
\toprule
 & $\epsilon$ & 1.00 & 0.90 & 0.80 & 0.70 & 0.60 & 0.50 & 0.40 & 0.30 & 0.20 & 0.10 & 0.05 & 0.00 \\
Dataset & Algorithm &  &  &  &  &  &  &  &  &  &  &  &  \\
\midrule
\multirow[t]{6}{*}{Aids2} & DRAFT & 1.04e+03 & 9.95e+02 & 9.48e+02 & 9.01e+02 & 8.54e+02 & 8.06e+02 & 7.56e+02 & 7.04e+02 & 6.49e+02 & 5.92e+02 & 5.66e+02 & 5.41e+02 \\
 & Noise & 9.94e+02 & 9.53e+02 & 9.11e+02 & 8.69e+02 & 8.26e+02 & 7.82e+02 & 7.37e+02 & 6.89e+02 & 6.39e+02 & 5.88e+02 & 5.64e+02 & 5.41e+02 \\
 & FGSM & 6.16e+02 & 6.10e+02 & 6.04e+02 & 5.97e+02 & 5.91e+02 & 5.84e+02 & 5.77e+02 & 5.70e+02 & 5.63e+02 & 5.54e+02 & 5.48e+02 & 5.41e+02 \\
 & PGD & 6.20e+02 & 6.13e+02 & 6.05e+02 & 5.98e+02 & 5.91e+02 & 5.83e+02 & 5.76e+02 & 5.68e+02 & 5.60e+02 & 5.52e+02 & 5.47e+02 & 5.41e+02 \\
 & AAE-Cox & 7.00e+02 & 6.83e+02 & 6.67e+02 & 6.51e+02 & 6.35e+02 & 6.19e+02 & 6.03e+02 & 5.87e+02 & 5.71e+02 & 5.55e+02 & 5.47e+02 & 5.39e+02 \\
 & SAWAR & 5.84e+02 & 5.80e+02 & 5.76e+02 & 5.

C:\Users\lpott\AppData\Local\Temp\ipykernel_440\1541890175.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(final_df.T.applymap(lambda x: str(np.round(x,3))).applymap(lambda x: "{:.2e}".format(float(x))).to_latex(index=True,multicolumn_format="c"))


## PDF PICTURES

In [ ]:
import matplotlib.pyplot as plt
import os
import glob
import numpy as np
# import pandas as pd
import re
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import pandas
import modin.pandas as pd

In [ ]:
def scrape_excel_data(root_folder,excel_name,seed_key):
    """
    Scrape data from Excel sheets in a nested folder structure into a dictionary.
    
    Args:
        root_folder (str): Root folder containing the data.
    
    Returns:
        dict: Nested dictionary where keys are datasets, sub-keys are algorithms,
              and sub-sub-keys are seeds, with values being DataFrames from the Excel sheets.
    """
    # Initialize the nested dictionary
    data_dict = {}

    # Traverse the root folder
    
    # Ensure it's a directory
    if os.path.isdir(root_folder):
        # Traverse the results folder for each attack method
        for algorithm_folder in tqdm(os.listdir(root_folder)):
            algorithm_path = os.path.join(root_folder, algorithm_folder)

            # Ensure it's a directory
            if os.path.isdir(algorithm_path):
                # Traverse the dataset folder for each algorithm
                for dataset_folder in os.listdir(algorithm_path):
                    dataset_path = os.path.join(algorithm_path, dataset_folder)

                    # Ensure it's a directory
                    if os.path.isdir(dataset_path):
                        # Initialize sub-dictionary for each dataset
                        if dataset_folder not in data_dict:
                            data_dict[dataset_folder] = {}

                        # Initialize sub-dictionary for each algorithm within the dataset
                        if algorithm_folder not in data_dict[dataset_folder]:
                            algo = algorithm_folder.split("_")[-1]

                            data_dict[dataset_folder][algo] = {}

                        # Traverse the seed folders
                        for seed_folder in os.listdir(dataset_path):
                            if seed_key not in seed_folder:
                                continue
                            seed_path = os.path.join(dataset_path, seed_folder)

                            # Ensure it's a directory and contains the Excel file
                            if os.path.isdir(seed_path):
                                excel_file_path = os.path.join(seed_path, excel_name)

                                if os.path.exists(excel_file_path):
                                    # Read the Excel file into a DataFrame
                                    # print(excel_file_path)
                                    df = pd.read_excel(excel_file_path,engine='openpyxl')

                                    # Extract seed identifier from the folder name
                                    seed_id = seed_folder.split('_')[-1]
                                    
                                    # Store the DataFrame in the nested dictionary
                                    data_dict[dataset_folder][algo][seed_id] = df

    return data_dict


In [ ]:
from tqdm import tqdm

In [ ]:
attack = "fgsm"
excel_name = "population_curves_attacked_test.xlsx"
seed_interest = "222"

base_path = osp.join(os.getcwd(),"results")
attack_folder =  f"attack_{attack}"
seeds = [str(i*111) for i in range(1,6)]
results_folder = os.path.join(base_path, attack_folder)

algorithms = ["draft","noise","fgsm","pgd","aae","crownibp"]
exclude_datasets = ["Dialysis","divorce","Pbc3","vlbw"]

In [ ]:
data_dict = scrape_excel_data(results_folder,excel_name,seed_interest)

In [ ]:
cwd = os.getcwd()
save_folder = osp.join(cwd,"results",f"attack_{attack}","perturb_curves.pdf")

In [ ]:
save_folder

n_rows = len(data_dict)
n_cols = len(data_dict["stagec"])

rename_dict = {"draft":"DRAFT","noise":"Noise","fgsm":"FGSM","pgd":"PGD","aae":"AAE-Cox","crownibp":"SAWAR"}

In [ ]:
from copy import deepcopy

In [ ]:
fig,axes = plt.subplots(n_rows,n_cols,figsize=(30,64),sharey=True)

SMALL_SIZE = 80
plt.rc('xtick', labelsize=SMALL_SIZE//2)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE//2)  
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SMALL_SIZE)     # fontsize of the x and y labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=SMALL_SIZE)   # fontsize of the figure title


for i,key_dataset in enumerate(data_dict.keys()):
    for j,key_al in enumerate(algorithms):
        print(i,j)


        df_population = deepcopy(data_dict[key_dataset][key_al][seed_interest])

        t = df_population.pop("t")
        base_models = df_population.iloc[:,:2]
        base_models.columns = [col.split("_")[0] for col in base_models.columns]
        base_models = base_models.rename(columns={"kmf":"KMC","St":"NN"})
        
        df_population= df_population.iloc[:,2:].iloc[:,-5:-1]
        
        df_population.columns = ["$\epsilon$={:.2f}".format(eval(col.split("=")[1])) for col in df_population.columns]

        # print(base_models)
        base1 = axes[i][j].plot(t,base_models.iloc[:,0],linewidth=3,c="b")
        base2 = axes[i][j].plot(t,base_models.iloc[:,1],linewidth=3,c="r")
        perturb1 = axes[i][j].plot(t,df_population,'--',linewidth=3)
  
        if j == 0:
            # base1 = axes[i][j].plot(t,base_models.iloc[:,0],linewidth=3,c="b")
            # base2 = axes[i][j].plot(t,base_models.iloc[:,1],linewidth=3,c="r")
            # perturb = axes[i][j].plot(t,df_population,'--',linewidth=3)

            axes[i][j].set_ylabel(f"{key_dataset}\n S(t)" ,fontsize=SMALL_SIZE//1.5)
            axes[i][j].set_xlabel("t",fontsize=SMALL_SIZE//1.5)

        axes[i][j].set_xlabel("t",fontsize=SMALL_SIZE//1.5)

for ax, col in zip(axes[0], algorithms):
    col = "SAWAR" if col == "crownibp" else col
    col = "DRAFT" if col == "draft" else col
    col = "PGD" if col == "pgd" else col
    col = "FGSM" if col == "fgsm" else col
    col = "Noise" if col == "noise" else col
    col = "AAE-Cox" if col == "aae" else col

    ax.set_title(col,fontsize=SMALL_SIZE//1.5)

labels = base_models.columns.tolist() + df_population.columns.tolist()
# labels[labels.index("baseline")] = "non-robust"

fig.legend([base1, base2,perturb1], labels=labels, 
           loc="upper center",ncols=6,fontsize=30,bbox_to_anchor=(.5,1.03),prop={'size':SMALL_SIZE//1.7}) 

# axes[0][3].legend(base_models.columns.tolist() + robust_df.columns.tolist(),fontsize=20,ncol=2,loc=1)

# plt.legend(base_models.columns.tolist() + robust_df.columns.tolist(),loc='upper center',ncol=5)
plt.tight_layout(pad=0)
plt.savefig(save_folder,dpi=1600,bbox_inches="tight")
plt.show()

### DIST PLOTS

In [ ]:
attack = "fgsm"
excel_name = "curve_distributions_test.xlsx"
seed_interest = "222"

base_path = osp.join(os.getcwd(),"results")
attack_folder =  f"attack_{attack}"
seeds = [str(i*111) for i in range(1,6)]
results_folder = os.path.join(base_path, attack_folder)

algorithms = ["draft","noise","fgsm","pgd","aae","crownibp"]
exclude_datasets = ["Dialysis","divorce","Pbc3","vlbw"]

In [ ]:
data_dict = scrape_excel_data(results_folder,excel_name,seed_interest)

In [ ]:
data_dict['Aids2']['draft']

In [ ]:
n_rows = len(data_dict)
n_cols = len(data_dict["stagec"])

rename_dict = {"draft":"DRAFT","noise":"Noise","fgsm":"FGSM","pgd":"PGD","aae":"AAE-Cox","crownibp":"SAWAR"}

In [ ]:
cwd = os.getcwd()
save_folder = osp.join(cwd,"results",f"attack_{attack}","dist_curves.pdf")

In [ ]:
import seaborn as sns

In [ ]:
fig,axes = plt.subplots(n_rows,n_cols,figsize=(30,64),sharey=True)

SMALL_SIZE = 80
plt.rc('xtick', labelsize=SMALL_SIZE//2)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE//2)  
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SMALL_SIZE)     # fontsize of the x and y labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=SMALL_SIZE)   # fontsize of the figure title


for i,key_dataset in enumerate(data_dict.keys()):
    for j,key_al in enumerate(algorithms):
        print(i,j)

        df_population = deepcopy(data_dict[key_dataset][key_al][seed_interest])

        t = df_population.pop("t")
                        
        mu = sns.lineplot(x=t, y=df_population.iloc[:,0], label='Average S(t)', linewidth=3.0, ax=axes[i][j],c='b',legend=False)
        q95 = sns.lineplot(x=t, y=df_population.iloc[:,1], label='Confidence', linewidth=3.0, ax=axes[i][j],c='r',legend=False)
        q05 = sns.lineplot(x=t, y=df_population.iloc[:,2], label='Confidence', linewidth=3.0, ax=axes[i][j],c='r',legend=False)

        line = q05.get_lines()
        axes[i][j].fill_between(line[0].get_xdata(), line[1].get_ydata(), line[2].get_ydata(), color='blue', alpha=.3)
        axes[i][j].set_xlabel("t",fontsize=SMALL_SIZE//1.5)

        if j == 0:
            axes[i][j].set_ylabel(f"{key_dataset}\n S(t)" ,fontsize=SMALL_SIZE//1.5)

for ax, col in zip(axes[0], algorithms):
    col = "SAWAR" if col == "crownibp" else col
    col = "DRAFT" if col == "draft" else col
    col = "PGD" if col == "pgd" else col
    col = "FGSM" if col == "fgsm" else col
    col = "Noise" if col == "noise" else col
    col = "AAE-Cox" if col == "aae" else col
    ax.set_title(col,fontsize=SMALL_SIZE//1.5)

labels = ["S(t)","Credible Interval","$Q_{95},Q_{05}$"]
fig.legend([mu,q95,q05], labels=labels, 
           loc="upper center",ncols=4,fontsize=30,bbox_to_anchor=(.5,1.05),prop={'size':SMALL_SIZE}) 

plt.tight_layout(pad=0)
plt.savefig(save_folder,dpi=1600,bbox_inches="tight")
plt.show()